In [10]:
# Install libraries
!pip install earthengine-api geemap --quiet

# Authenticate and initialize Earth Engine with public project
import ee

ee.Authenticate()
ee.Initialize(project='semiotic-summer-461404-d1')



In [25]:
# Define a bounding box over a part of the Amazon rainforest
amazon_region = ee.Geometry.BBox(-63.0, -9.0, -60.0, -6.0)  # Rondônia, Brazil


In [26]:
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')  # ✅ Updated band names
    return image.addBands(ndvi)



In [43]:
# Filter for cloud-free images during dry season (July to Sept)
def get_landsat_ndvi(year):
    collection = (
        ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .filterBounds(amazon_region)
        .filterDate(f'{year}-06-01', f'{year}-10-31')  # 👈 extended range
        .filterMetadata('CLOUD_COVER', 'less_than', 40)  # 👈 allow more
        .map(calculate_ndvi)
    )
    return collection.median().clip(amazon_region)



ndvi_2014 = get_landsat_ndvi(2014).select('NDVI')
ndvi_2024 = get_landsat_ndvi(2022).select('NDVI')  # Use 2022 as 2024 data may be incomplete


In [44]:
# Difference between 2024 and 2014 NDVI
ndvi_diff = ndvi_2024.subtract(ndvi_2014).rename('NDVI_Diff')

# Binary mask: vegetation loss (drop > 0.2)
loss_mask = ndvi_diff.lt(-0.2).rename('Deforestation_Mask')


In [46]:
# Set map center and initialize map
Map = geemap.Map(center=[-7.5, -61.5], zoom=7)

# Visualization params
ndvi_params = {'min': 0, 'max': 1, 'palette': ['white', 'green']}
diff_params = {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'green']}
mask_params = {'min': 0, 'max': 1, 'palette': ['black', 'yellow']}

# Add layers
Map.addLayer(ndvi_2014, ndvi_params, 'NDVI 2014')
Map.addLayer(ndvi_2024, ndvi_params, 'NDVI 2022')
Map.addLayer(ndvi_diff, diff_params, 'NDVI Difference')
Map.addLayer(loss_mask, mask_params, 'Deforestation Mask')

Map.addLayerControl()
Map


Map(center=[-7.5, -61.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…